In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"patrick2544","key":"c3229e80d88bf54d692e52ac5ebd32ac"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d dmitrybabko/speech-emotion-recognition-en

Dataset URL: https://www.kaggle.com/datasets/dmitrybabko/speech-emotion-recognition-en
License(s): copyright-authors


In [ ]:
!unzip speech-emotion-recognition-en.zip -d speech_emotion_data

Streaming output truncated to the last 5000 lines.
  inflating: speech_emotion_data/Crema/1088_IWW_FEA_XX.wav  
  inflating: speech_emotion_data/Crema/1088_IWW_HAP_XX.wav  
  inflating: speech_emotion_data/Crema/1088_IWW_NEU_XX.wav  
  inflating: speech_emotion_data/Crema/1088_IWW_SAD_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_ANG_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_DIS_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_FEA_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_HAP_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_NEU_XX.wav  
  inflating: speech_emotion_data/Crema/1088_MTI_SAD_XX.wav  
  inflating: speech_emotion_data/Crema/1088_TAI_ANG_XX.wav  
  inflating: speech_emotion_data/Crema/1088_TAI_DIS_XX.wav  
  inflating: speech_emotion_data/Crema/1088_TAI_FEA_XX.wav  
  inflating: speech_emotion_data/Crema/1088_TAI_HAP_XX.wav  
  inflating: speech_emotion_data/Crema/1088_TAI_NEU_XX.wav  
  inflating: speech_emotion_data/C

In [ ]:
!ls speech_emotion_data/

Crema  Ravdess	Savee  Tess


In [ ]:
import os
import librosa

dataset_path = "speech_emotion_data/"

for file in os.listdir(dataset_path):
    if file.endswith(".wav"):
        file_path = os.path.join(dataset_path, file)
        audio, sr = librosa.load(file_path, sr=None)
        print(f"Loaded {file} | Duration: {len(audio)/sr:.2f} sec | Sample Rate: {sr} Hz")

In [ ]:
dataset_path = "speech_emotion_data/"  # Change this if your folder has a different name

# List the files and folders inside the dataset
print("Files in dataset folder:")
print(os.listdir(dataset_path))

Files in dataset folder:
['Savee', 'Ravdess', 'Tess', 'Crema']


In [ ]:
emotion_map = {
    "neutral": "neutral", "calm": "neutral",  # Merge calm into neutral if needed
    "happy": "happy", "hap": "happy",
    "sad": "sad", "sa": "sad",
    "angry": "angry", "ang": "angry",
    "fearful": "fearful", "fea": "fearful",
    "disgust": "disgust", "dis": "disgust",
    "surprised": "surprised", "su": "surprised", "surprise": "surprised"
}

In [ ]:
def get_label_from_filename(dataset, filename):
    """Extracts emotion label based on dataset-specific rules."""

    if dataset == "Ravdess":
        ravdess_emotions = {
            "01": "neutral", "02": "calm", "03": "happy", "04": "sad",
            "05": "angry", "06": "fearful", "07": "disgust", "08": "surprised"
        }
        parts = filename.split("-")
        if len(parts) > 2:
            return ravdess_emotions.get(parts[2], "unknown")

    elif dataset == "Crema":
        parts = filename.split("_")
        if len(parts) > 2:
            return parts[2].lower()  # SAD → sad, HAP → happy, etc.

    elif dataset == "Tess":
        return filename.split("_")[-1].replace(".wav", "").lower()

    elif dataset == "Savee":
        savee_emotions = {
            "a": "angry", "d": "disgust", "f": "fearful", "h": "happy",
            "n": "neutral", "sa": "sad", "su": "surprised"
        }
        # Extract emotion code by skipping speaker ID (e.g., "JK_a10.wav" → "a10")
        emotion_code = filename.split("_")[-1][:2]  # Extract first two characters after '_'
        if emotion_code not in savee_emotions:
            emotion_code = emotion_code[0]  # If the second letter is a number, take only first letter
        return savee_emotions.get(emotion_code, "unknown")

    return "unknown"

In [ ]:
import librosa
import numpy as np
import pandas as pd

dataset_path = "speech_emotion_data/"

# Store features and labels
features = []
labels = []

for dataset in ["Savee", "Ravdess", "Tess", "Crema"]:
    folder_path = os.path.join(dataset_path, dataset)

    for file in os.listdir(folder_path):
        if file.endswith(".wav"):
            file_path = os.path.join(folder_path, file)

            # Extract the correct label
            raw_label = get_label_from_filename(dataset, file)
            standardized_label = emotion_map.get(raw_label, "unknown")

            # Load the audio file
            audio, sr = librosa.load(file_path, sr=None)

            # Extract MFCC features
            mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40).T, axis=0)

            # Store features and labels
            features.append(mfccs)
            labels.append(standardized_label)

# Convert to DataFrame
df = pd.DataFrame(features)
df["emotion"] = labels

In [ ]:
print(df["emotion"].value_counts())  # Count of each emotion in the dataset
df.head()  # Show the first few rows

emotion
sad          1331
angry        1331
disgust      1331
happy        1331
fearful      1331
unknown      1087
neutral       120
surprised      60
Name: count, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-405.993896,101.260620,19.537207,24.806614,29.927975,16.815874,5.344127,-10.670869,-5.648187,4.944284,...,-2.157680,0.284405,-0.000149,-1.631652,0.009484,1.443457,1.591549,2.631166,4.486713,sad
1,-252.977097,130.545227,-4.214458,30.715477,21.104128,-10.135195,-6.344809,-11.809314,-7.099907,-4.221438,...,-1.101522,2.685668,2.825537,1.442364,2.712428,2.156740,1.864284,3.013726,2.604978,angry
2,-661.175964,126.117920,43.181236,36.686943,29.435568,8.211150,1.062881,-4.409803,-1.835169,3.735992,...,2.761153,1.778863,0.554426,-0.124815,2.076975,3.833216,2.136405,0.530403,1.337839,disgust
3,-396.929230,101.259132,27.250980,30.957096,15.131433,-3.888555,2.324689,-4.695027,-8.689311,-2.614920,...,-0.593944,2.452221,2.420918,0.394558,1.797144,2.905242,1.829163,3.281039,5.341721,disgust
4,-355.878448,117.681931,20.387419,36.800877,27.563715,4.279939,-0.885260,-14.275339,-5.615031,2.700219,...,0.209984,0.711453,0.083018,0.635643,1.358058,1.238535,2.636079,3.930555,4.082411,happy


In [ ]:
from sklearn.preprocessing import StandardScaler

# Extract only feature columns (excluding the emotion label)
X = df.iloc[:, :-1]  # All columns except the last one (features)
y = df["emotion"]  # The last column (labels)

# Apply StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame
df_scaled = pd.DataFrame(X_scaled, columns=df.columns[:-1])
df_scaled["emotion"] = y  # Re-add labels

In [ ]:
import pickle

# Save as CSV
df_scaled.to_csv("final_preprocessed_data.csv", index=False)

# Save as Pickle file
with open("final_preprocessed_data.pkl", "wb") as f:
    pickle.dump(df_scaled, f)

print("✅ Final preprocessed data saved successfully!")

✅ Final preprocessed data saved successfully!


In [ ]:
# Load CSV file
df_csv = pd.read_csv("final_preprocessed_data.csv")
print("CSV Data Loaded:", df_csv.shape)

# Load Pickle file
with open("final_preprocessed_data.pkl", "rb") as f:
    df_pkl = pickle.load(f)

print("Pickle Data Loaded:", df_pkl.shape)

CSV Data Loaded: (7922, 41)
Pickle Data Loaded: (7922, 41)


In [ ]:
from google.colab import files

# Download CSV file
files.download("final_preprocessed_data.csv")

# Download Pickle file
files.download("final_preprocessed_data.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>